In [2]:
!pip install simpy


  Using cached simpy-4.1.1-py3-none-any.whl (27 kB)


In [45]:
#importing important libraries

In [46]:
import simpy
import random

In [47]:
#defining required parameters

In [48]:
avg_arrival_time = 5  # average interval between vessel arrivals in hours
containers_per_vessel = 150
crane_operation_time = 3  # time taken by a crane to unload a container in minutes
truck_operation_time = 6  # time taken by a truck to transport a container minutes
num_berths = 2
num_cranes = 2
num_trucks = 3
simulation_time = 1000  # total simulation time in minutes

In [49]:
# Define the Terminal class

In [50]:
class Terminal:
    def __init__(self, env, num_berths, num_cranes, num_trucks):
        self.env = env
        self.berths = simpy.Resource(env, num_berths)
        self.cranes = simpy.Resource(env, num_cranes)
        self.trucks = simpy.Resource(env, num_trucks)
        
    def unload_container(self, vessel_name, container_id):
        yield self.env.timeout(crane_operation_time)
        print(f'{self.env.now:.2f}: Crane has unloaded container {container_id} from {vessel_name}')

        with self.trucks.request() as truck_request:
            yield truck_request
            print(f'{self.env.now:.2f}: Truck is transporting container {container_id} from {vessel_name}')
            yield self.env.timeout(truck_operation_time)
            print(f'{self.env.now:.2f}: Truck has delivered container {container_id} from {vessel_name}')


In [51]:
# Define the vessel process

In [59]:
def vessel(env, name, terminal):
    print(f'{env.now:.2f}: {name} has arrived')

    with terminal.berths.request() as request:
        yield request
        print(f'{env.now:.2f}: {name} is berthing')

        for container in range(1, containers_per_vessel + 1):
            with terminal.cranes.request() as crane_request:
                yield crane_request
                print(f'{env.now:.2f}: Crane is unloading container {container} from {name}')
                yield env.process(terminal.unload_container(name, container))

        print(f'{env.now:.2f}: {name} has departed')

In [60]:
# Define the vessel arrivals process

In [61]:
def vessel_arrivals(env, terminal):
    vessel_id = 0
    while True:
        inter_arrival_time = random.expovariate(1.0 / avg_arrival_time)
        yield env.timeout(inter_arrival_time * 60)  # convert hours to minutes
        vessel_id += 1
        env.process(vessel(env, f'Vessel {vessel_id}', terminal))

In [62]:
# Setup the simulation
print('Container Terminal Simulation')
random.seed(42)  # seed for reproducibility

Container Terminal Simulation


In [63]:
# Create the environment and terminal
env = simpy.Environment()
terminal = Terminal(env, num_berths, num_cranes, num_trucks)
env.process(vessel_arrivals(env, terminal))

<Process(vessel_arrivals) object at 0x1702fe899a0>

In [64]:
# Run the simulation
env.run(until=simulation_time)

306.02: Vessel 1 has arrived
306.02: Vessel 1 is berthing
306.02: Crane is unloading container 1 from Vessel 1
309.02: Crane has unloaded container 1 from Vessel 1
309.02: Truck is transporting container 1 from Vessel 1
313.62: Vessel 2 has arrived
313.62: Vessel 2 is berthing
313.62: Crane is unloading container 1 from Vessel 2
315.02: Truck has delivered container 1 from Vessel 1
315.02: Crane is unloading container 2 from Vessel 1
316.62: Crane has unloaded container 1 from Vessel 2
316.62: Truck is transporting container 1 from Vessel 2
318.02: Crane has unloaded container 2 from Vessel 1
318.02: Truck is transporting container 2 from Vessel 1
322.62: Truck has delivered container 1 from Vessel 2
322.62: Crane is unloading container 2 from Vessel 2
324.02: Truck has delivered container 2 from Vessel 1
324.02: Crane is unloading container 3 from Vessel 1
325.62: Crane has unloaded container 2 from Vessel 2
325.62: Truck is transporting container 2 from Vessel 2
327.02: Crane has unl